In [296]:
%load_ext sql
%sql ibm_db_sa://bzg64264:de1zuazffWjooMLP@764264db-9824-4b7c-82df-40d1b13897c2.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:32536/bludb?security=SSL
%config SqlMagic.displaycon = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [257]:
%%sql
select s.name_of_school, s.COMMUNITY_AREA_NAME, s.AVERAGE_STUDENT_ATTENDANCE, c.HARDSHIP_INDEX
from CHICAGO_PUBLIC_SCHOOLS_DATA as s left join CENSUS_DATA as c
on c.COMMUNITY_AREA_NUMBER=s.COMMUNITY_AREA_NUMBER
where HARDSHIP_INDEX = 98
;

 * ibm_db_sa://bzg64264:***@764264db-9824-4b7c-82df-40d1b13897c2.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:32536/bludb?security=SSL
Done.


name_of_school,community_area_name,average_student_attendance,hardship_index
George Washington Carver Military Academy High School,RIVERDALE,91.60%,98
George Washington Carver Primary School,RIVERDALE,90.90%,98
Ira F Aldridge Elementary School,RIVERDALE,92.90%,98
William E B Dubois Elementary School,RIVERDALE,93.30%,98


In [17]:
%%sql
select c.case_number, c.primary_type, c.community_area_number,  v.community_area_name
from CHICAGO_CRIME_DATA as c left outer join CENSUS_DATA as v
on c.community_area_number=v.community_area_number
where lower(c.location_description) like '%school%'
;

 * ibm_db_sa://bzg64264:***@764264db-9824-4b7c-82df-40d1b13897c2.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:32536/bludb?security=SSL
Done.


case_number,primary_type,community_area_number,community_area_name
HK577020,NARCOTICS,1,Rogers Park
HL725506,BATTERY,4,Lincoln Square
HH639427,BATTERY,25,Austin
HS200939,CRIMINAL DAMAGE,25,Austin
HT315369,ASSAULT,27,East Garfield Park
HP716225,BATTERY,35,Douglas
HL353697,BATTERY,43,South Shore
HS305355,NARCOTICS,58,Brighton Park
JA460432,BATTERY,70,Ashburn
HR585012,CRIMINAL TRESPASS,70,Ashburn


# Views

In [259]:
%%sql 
select colname from syscat.columns 
where LOWER(tabname)='obsview'
;

 * ibm_db_sa://bzg64264:***@764264db-9824-4b7c-82df-40d1b13897c2.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:32536/bludb?security=SSL
Done.


colname
ENVIRONMENT_RATING
FAMILY_RATING
INSTRUCTION_RATING
LEADERS_RATING
SAFETY_RATING
SCHOOL_NAME
TEACHERS_RATING


In [258]:
%%sql
create or replace view obsview
  (School_Name, Safety_Rating,Family_Rating,Environment_Rating,Instruction_Rating, Leaders_Rating, Teachers_Rating) 
as select NAME_OF_SCHOOL,Safety_Icon,Family_Involvement_Icon,Environment_Icon,Instruction_Icon,Leaders_Icon,Teachers_Icon
from CHICAGO_PUBLIC_SCHOOLS_DATA
;

 * ibm_db_sa://bzg64264:***@764264db-9824-4b7c-82df-40d1b13897c2.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:32536/bludb?security=SSL
Done.


[]

In [261]:
%%sql
select SCHOOL_NAME, LEADERS_RATING from obsview
limit 5
;


 * ibm_db_sa://bzg64264:***@764264db-9824-4b7c-82df-40d1b13897c2.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:32536/bludb?security=SSL
Done.


school_name,leaders_rating
Abraham Lincoln Elementary School,Weak
Adam Clayton Powell Paideia Community Academy Elementary School,Weak
Adlai E Stevenson Elementary School,Weak
Agustin Lara Elementary Academy,Weak
Air Force Academy High School,Weak


In [ ]:
%%sql 
--#SET TERMINATOR @
CREATE OR REPLACE PROCEDURE 
UPDATE_LEADERS_SCORE (IN in_School_ID INT, IN in_Leader_Score INT)
LANGUAGE SQL
MODIFIES SQL DATA

BEGIN
    UPDATE CHICAGO_PUBLIC_SCHOOLS_DATA   
    SET Leaders_Score = in_Leader_Score    
    WHERE SCHOOL_ID = in_School_ID;
    
    IF in_Leader_Score  < 20 AND in_Leader_Score >= 0 THEN  UPDATE CHICAGO_PUBLIC_SCHOOLS_DATA  
        SET LEADERS_ICON = 'Very weak'  WHERE SCHOOL_ID = in_School_ID;
    ELSEIF in_Leader_Score  < 40 THEN  UPDATE CHICAGO_PUBLIC_SCHOOLS_DATA  
        SET LEADERS_ICON = 'Weak'  WHERE SCHOOL_ID = in_School_ID;
    ELSEIF in_Leader_Score  < 60 THEN   UPDATE CHICAGO_PUBLIC_SCHOOLS_DATA  
        SET LEADERS_ICON = 'Average'  WHERE SCHOOL_ID = in_School_ID;
    ELSEIF in_Leader_Score  < 80 THEN   UPDATE CHICAGO_PUBLIC_SCHOOLS_DATA  
        SET LEADERS_ICON = 'Strong'  WHERE SCHOOL_ID = in_School_ID;
    ELSEIF in_Leader_Score  < 100 THEN   UPDATE CHICAGO_PUBLIC_SCHOOLS_DATA  
        SET LEADERS_ICON = 'Very strong'    WHERE SCHOOL_ID = in_School_ID;
    ELSE
        ROLLBACK WORK;
    END IF;
	COMMIT WORK;
END
@

In [312]:
%sql CALL UPDATE_LEADERS_SCORE(609674, 38);
%sql SELECT NAME_OF_SCHOOL,LEADERS_SCORE,LEADERS_ICON from CHICAGO_PUBLIC_SCHOOLS_DATA  where school_id = 609674;

Done.
Done.


name_of_school,leaders_score,leaders_icon
Chicago Vocational Career Academy High School,38,Weak


In [313]:
%sql CALL UPDATE_LEADERS_SCORE(609674, 101);
%sql SELECT NAME_OF_SCHOOL,LEADERS_SCORE,LEADERS_ICON from CHICAGO_PUBLIC_SCHOOLS_DATA  where school_id = 609674;

Done.
Done.


name_of_school,leaders_score,leaders_icon
Chicago Vocational Career Academy High School,38,Weak


In [128]:
%%sql
ALTER TABLE CHICAGO_PUBLIC_SCHOOLS_DATA 
ALTER COLUMN leaders_icon SET DATA TYPE varchar(15);

 * ibm_db_sa://bzg64264:***@764264db-9824-4b7c-82df-40d1b13897c2.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:32536/bludb?security=SSL
Done.


[]